In [93]:
import tweepy as tw
import pandas as pd
from datetime import datetime, date, timedelta
import json

In [94]:
with open(r'D:\Estudos\FIAP\Acessos.json', 'r') as arquivo:
    acesso_api =  json.load(arquivo)

In [95]:
cliente = tw.Client(bearer_token=acesso_api['bearer_token'], 
                    consumer_key=acesso_api['consumer_key'], 
                    consumer_secret=acesso_api['consumer_secret'],
                    access_token=acesso_api['access_token'],
                    access_token_secret=acesso_api['access_token_secret'])

In [107]:
def busca_tweet(data_inicio, data_fim, intervalo,query):
    df_i = datetime.fromisoformat(data_inicio)
    df_f = datetime.fromisoformat(data_fim)

    lista_dados = []
    while (df_i < df_f):
        start = str(df_i).replace(' ','T')+'Z'
        df_i = df_i+timedelta(minutes=intervalo)-timedelta(seconds=1)
        end = str(df_i).replace(' ','T')+'Z'
        df_i = df_i+timedelta(seconds=1)
        lista_tweets = cliente.search_recent_tweets(query=query, 
                                                    max_results=100,
                                                    start_time=start,
                                                    end_time=end, 
                                                    tweet_fields=['created_at',
                                                                    'public_metrics',
                                                                    'lang',
                                                                    'author_id',
                                                                    'in_reply_to_user_id'
                                                                    ])
        lista_dados.append(lista_tweets.data)
    return lista_dados
        

In [108]:
lista_dados = busca_tweet('2022-03-25','2022-03-26',10,'studio_z')

In [161]:
resultado = []
for tweet_data in lista_dados:

    if tweet_data is not None and len(tweet_data) > 0:
        for tweet in tweet_data:
            obj = {
                    "id":tweet.id,
                    "author_id":tweet.author_id,
                    "created_at":str(tweet.created_at)[:19],
                    "text":tweet.text,
                    "lang":tweet.lang,
                    "in_reply_to_user_id":tweet.in_reply_to_user_id,
                    "public_metrics":
                    {
                        "retweet_count":tweet.public_metrics['retweet_count'],
                        "reply_count":tweet.public_metrics['reply_count'],
                        "like_count":tweet.public_metrics['like_count'],
                        "quote_count":tweet.public_metrics['quote_count']
                    }
                }
            resultado.append(obj)

In [164]:
with open('D:\Estudos\FIAP\Fase 5\dados.json', 'w') as arquivo:
    arquivo.write(json.dumps(resultado, indent=4))